<a href="https://colab.research.google.com/github/r-dube/fakejobs/blob/main/fj_composite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Load the modules used
import numpy as np
import pandas as pd
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, roc_auc_score
from keras.models import Model
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, GlobalMaxPool1D, Conv1D, MaxPool1D, Flatten, RepeatVector, Input, Embedding, Concatenate
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import Constant 
from keras.optimizers import Adam
from keras import metrics
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from collections import Counter

In [2]:
# NLTK to remove stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# For reproducible results
import random as rn
import os
import tensorflow as tf
os.environ['PYTHONHASHSEED'] = '42'
os.environ['CUDA_VISIBLE_DEVICES'] = ''
np.random.seed(42)
rn.seed(42)
tf.random.set_seed(42)

In [4]:
# Set data_url, the location of the data
# Data is not loaded from a local file
# data_url="https://raw.githubusercontent.com/r-dube/fakejobs/main/data/fj_small.csv"
# data_url="https://raw.githubusercontent.com/r-dube/fakejobs/main/data/fj_medium.csv"
data_url="https://raw.githubusercontent.com/r-dube/fakejobs/main/data/fake_job_postings.csv"

In [5]:
def fj_load_df_from_url():
    """
    Load dataframe from csv file
    Input:
        None
    Returns:
        dataframe
    """

    df = pd.read_csv(data_url)

    print ('Loaded dataframe shape', df.shape)

    counts = fj_label_stats(df)
    print ('Not fraudulent', counts[0], 'Fraudulent', counts[1])

    print(df.describe())

    print ('NAs/NANs in data =>')
    print(df.isna().sum())

    return df

def fj_label_stats(df):
    """
    Very basic label statistics
    Input: 
        Dataframe
    Returns:
        Number of samples with 0, 1 as the label
    """
    counts = np.bincount(df['fraudulent'])
    return counts

def fj_txt_only(df):
    """
    Combine all the text fields, discard everything else except for the label
    Input: 
        Dataframe
    Returns:
        Processed dataframe
    """
    
    df.fillna(" ", inplace = True)

    df['text'] = df['title'] + ' ' + df['location'] + ' ' + df['department'] + \
    ' ' + df['company_profile'] + ' ' + df['description'] + ' ' + \
    df['requirements'] + ' ' + df['benefits'] + ' ' + df['employment_type'] + \
    ' ' + df['required_education'] + ' ' + df['industry'] + ' ' + df['function'] 

    del df['title']
    del df['location']
    del df['department']
    del df['company_profile']
    del df['description']
    del df['requirements']
    del df['benefits']
    del df['employment_type']
    del df['required_experience']
    del df['required_education']
    del df['industry']
    del df['function']  
    
    del df['salary_range']
    del df['job_id']
    del df['telecommuting']
    del df['has_company_logo']
    del df['has_questions']

    return df

In [6]:
df = fj_load_df_from_url()
df = fj_txt_only(df)
print('Maximum text length', df['text'].str.len().max())

Loaded dataframe shape (17880, 18)
Not fraudulent 17014 Fraudulent 866
             job_id  telecommuting  ...  has_questions    fraudulent
count  17880.000000   17880.000000  ...   17880.000000  17880.000000
mean    8940.500000       0.042897  ...       0.491723      0.048434
std     5161.655742       0.202631  ...       0.499945      0.214688
min        1.000000       0.000000  ...       0.000000      0.000000
25%     4470.750000       0.000000  ...       0.000000      0.000000
50%     8940.500000       0.000000  ...       0.000000      0.000000
75%    13410.250000       0.000000  ...       1.000000      0.000000
max    17880.000000       1.000000  ...       1.000000      1.000000

[8 rows x 5 columns]
NAs/NANs in data =>
job_id                     0
title                      0
location                 346
department             11547
salary_range           15012
company_profile         3308
description                1
requirements            2695
benefits                7210
telec

In [7]:
# Utilities to clean text

def remove_URL(text):
    url = re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r"", text)

def remove_html(text):
    html = re.compile(r"<.*?>")
    return html.sub(r"", text)

def remove_emoji(string):
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        "]+",
        flags=re.UNICODE,
    )
    return emoji_pattern.sub(r"", string)

def remove_punct(text):
    table = str.maketrans("", "", string.punctuation)
    return text.translate(table)

In [8]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
stop = set(stopwords.words("english"))

def remove_stopwords(text):
    text = [word.lower() for word in text.split() if word.lower() not in stop]

    return " ".join(text)

In [10]:
# clean text
df['text'] = df['text'].map(lambda x: remove_URL(x))
df['text'] = df['text'].map(lambda x: remove_html(x))
df['text'] = df['text'].map(lambda x: remove_emoji(x))
df['text'] = df['text'].map(lambda x: remove_punct(x))
df['text'] = df["text"].map(remove_stopwords)

In [11]:
# Count unique words
def counter_word(text):
    count = Counter()
    for i in text.values:
        for word in i.split():
            count[word] += 1
    return count

text = df['text']

counter = counter_word(text)
num_words = len(counter)
print ("Number of words: ", num_words)

# Max number of words in a sequence
max_length = 250

Number of words:  172475


In [12]:
# train-test split
train_text, test_text, train_labels , test_labels = train_test_split(df['text'], df['fraudulent'] , test_size = 0.15)
print(train_text.shape)

(15198,)


In [13]:
# Token Model

# max_num_words variable in case we want to clip the number of words
max_num_words = num_words

tokenizer = Tokenizer(num_words=max_num_words)
tokenizer.fit_on_texts(train_text)

word_index = tokenizer.word_index

train_sequences = tokenizer.texts_to_sequences(train_text)
train_padded = pad_sequences(
    train_sequences, maxlen=max_length, padding="post", truncating="post"
)

test_sequences = tokenizer.texts_to_sequences(test_text)
test_padded = pad_sequences(
    test_sequences, maxlen=max_length, padding="post", truncating="post"
)

print(f"Shape of train {train_padded.shape}")
print(f"Shape of test {test_padded.shape}")

Shape of train (15198, 250)
Shape of test (2682, 250)


In [14]:
# Char model

# Assuming that the text has been processed for word tokenization
# alphabet="abcdefghijklmnopqrstuvwxyz0123456789-,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]{}"
alphabet="abcdefghijklmnopqrstuvwxyz0123456789"
encoding_size = len(alphabet)
char_input_size=1000
dict = {}  # Maps each character to an integer
for idx, char in enumerate(alphabet):
  dict[char] = idx + 1

print(encoding_size)
print (dict)

36
{'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26, '0': 27, '1': 28, '2': 29, '3': 30, '4': 31, '5': 32, '6': 33, '7': 34, '8': 35, '9': 36}


In [15]:
# Char model

# return one-hot-vector from string
def str_to_ohv(s, ohv):
  max_length = min(len(s), char_input_size)
  for i in range(0, max_length):
    c = s[i]
    if c in dict:
      ohv[i, dict[c]-1] = 1
  return ohv

In [16]:
# Char model

num_jobs = train_text.shape[0]
train_t = np.zeros((num_jobs, char_input_size, encoding_size), dtype=np.int8)
i = 0
for _, val in train_text.iteritems():
  str_to_ohv(val, train_t[i])
  i=i+1

In [17]:

embed_dim = 32

# Word (TOKEN) model
token_input = Input(shape=(max_length,))
token_embedding = Embedding(num_words, embed_dim, input_length=max_length)(token_input)
token_conv = Conv1D(64, kernel_size=3, strides=1, padding="valid", activation="relu")(token_embedding)
token_pool = MaxPool1D(pool_size=3, strides=3)(token_conv)
token_drop = Dropout(.5)(token_pool)

# CHAR model
char_input = Input(shape=(char_input_size, encoding_size))
char_conv = Conv1D(64, kernel_size=3, strides=1, padding="valid",activation="relu")(char_input)
char_pool = GlobalMaxPool1D()(char_conv)
char_drop = Dropout(.5)(char_pool)
char_repeated = RepeatVector(2)(char_drop)

# Merge
merged = Concatenate(axis=1)([token_drop, char_repeated])
lstm = Bidirectional(LSTM(32, dropout=0.3, recurrent_dropout=0.01))(merged)
output = Dense(1, activation="sigmoid")(lstm)

# define a model with a list of two inputs
model = Model(inputs=[token_input, char_input], outputs=output)

In [18]:
model.compile(optimizer = 'adam' , loss = 'binary_crossentropy' , metrics = ['accuracy', tf.keras.metrics.FalsePositives(), tf.keras.metrics.FalseNegatives()])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 250)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 1000, 36)]   0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 250, 32)      5519200     input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 998, 64)      6976        input_2[0][0]                    
______________________________________________________________________________________________

In [19]:
model.fit([train_padded, train_t], train_labels, epochs = 7)

Epoch 1/7
475/475 [==============================] - 106s 211ms/step - loss: 0.2033 - accuracy: 0.9342 - false_positives: 61.4286 - false_negatives: 297.6912
Epoch 2/7
475/475 [==============================] - 99s 209ms/step - loss: 0.0275 - accuracy: 0.9921 - false_positives: 24.8887 - false_negatives: 33.7752
Epoch 3/7
475/475 [==============================] - 100s 211ms/step - loss: 0.0079 - accuracy: 0.9979 - false_positives: 7.2395 - false_negatives: 9.1954
Epoch 4/7
475/475 [==============================] - 100s 210ms/step - loss: 0.0028 - accuracy: 0.9993 - false_positives: 1.7815 - false_negatives: 4.2059
Epoch 5/7
475/475 [==============================] - 99s 208ms/step - loss: 8.4348e-04 - accuracy: 0.9999 - false_positives: 0.4832 - false_negatives: 0.9496
Epoch 6/7
475/475 [==============================] - 99s 209ms/step - loss: 0.0015 - accuracy: 0.9997 - false_positives: 1.5483 - false_negatives: 0.7080
Epoch 7/7
475/475 [==============================] - 99s 208ms/s

In [20]:
num_jobs = test_text.shape[0]
test_t = np.zeros((num_jobs, char_input_size, encoding_size), dtype=np.int8)
i = 0
for _, val in test_text.iteritems():
  str_to_ohv(val, test_t[i])
  i=i+1

In [21]:
pred_soft = model.predict([test_padded, test_t])

In [22]:
# pred = np.around(pred_soft, decimals = 0)
pred = np.where(pred_soft > 0.15, 1, 0)

acc = accuracy_score(pred, test_labels)
f1 = f1_score(pred, test_labels)

cm = confusion_matrix(test_labels, pred)
tn = cm[0][0]
fn = cm[1][0]
tp = cm[1][1]
fp = cm[0][1]

print('Accuracy score: {:.4f}'.format(acc), 'F1 score: {:.4f}'.format(f1))
print('False Positives: {:.0f}'.format(fp), 'False Negatives: {:.0f}'.format(fn))
print('Confusion matrix:\n', cm)

Accuracy score: 0.9843 F1 score: 0.8409
False Positives: 9 False Negatives: 33
Confusion matrix:
 [[2529    9]
 [  33  111]]


In [23]:
auc = roc_auc_score(test_labels, pred_soft)
print('AUC score: {:.4f}'.format(auc))

AUC score: 0.9707
